In [1]:
# --- 0. CONFIGURAÇÃO DO AMBIENTE ---
import sys
import os
import json 
import re 
import unidecode 
from pathlib import Path
import warnings

notebook_dir = os.getcwd() 
PROJECT_ROOT_PATH = os.path.abspath(os.path.join(notebook_dir, '..'))
SRC_PATH = os.path.join(PROJECT_ROOT_PATH, "src")
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)
    print(f"Adicionado ao sys.path: {SRC_PATH}")

warnings.filterwarnings("ignore", category=UserWarning, module='huggingface_hub')

try:
    from rag_pipeline.loader import load_and_process_jsons
    from rag_pipeline.vector_store import get_embedding_model, get_vector_store
    from rag_pipeline.model_setup import get_llm
    from rag_pipeline.chain import create_rag_chain
except ImportError as e:
    print(f"!!! ERRO DE IMPORTAÇÃO !!! Falha ao importar módulos da 'src/rag_pipeline'.")
    print(f"Verifique se o PROJECT_ROOT_PATH está correto: {PROJECT_ROOT_PATH}")
    print(f"Erro: {e}")
    raise e

# --- LÓGICA DE NORMALIZAÇÃO DE PERGUNTA ---
def carregar_dicionarios(input_dir: str) -> dict[str, dict[str, str]]:
    """Carrega os dicionários de normalização."""
    dictionaries_path = os.path.join(input_dir, 'dicionarios.json')
    try:
        with open(dictionaries_path, 'r', encoding='utf-8') as f:
            dictionaries = json.load(f)
        
        acronyms = {k.lower(): v.lower() for k, v in dictionaries.get("acronyms", {}).items()}
        standardization = {k.lower(): v.lower() for k, v in dictionaries.get("standardization_map", {}).items()}
        
        print("-> Dicionários de normalização carregados para o Testador RAG.")
        return {"acronyms": acronyms, "standardization": standardization}
    except FileNotFoundError:
        print(f"-> AVISO: 'dicionarios.json' não encontrado. Usando dicionários vazios.")
        return {"acronyms": {}, "standardization": {}}

def normalize_query(text: str, acronyms_map: dict, standardization_map: dict) -> str:
    """Aplica a mesma normalização dos dados à pergunta do usuário."""
    if not isinstance(text, str):
        return ""
    text = text.lower()  
    text = unidecode.unidecode(text)
    
    # Ordem: Siglas primeiro, depois Padronização
    for acronym, expansion in acronyms_map.items():
        replacement_string = f"{acronym} {expansion}"
        text = re.sub(r'\b' + re.escape(acronym) + r'\b', replacement_string, text)
        
    for key, value in standardization_map.items():
        text = re.sub(r'\b' + re.escape(key) + r'\b', value, text)

    text = re.sub(r'[^a-z0-9\s.,-]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# --- FUNÇÃO DE LIMPEZA DE RESPOSTA ---
def clean_model_response(response: str) -> str:
    """Limpa o 'eco' do prompt e remove duplicações do texto normalizado."""
    
    match = re.search(r'<start_of_turn>model\s*(.*)', response, re.DOTALL)
    if match:
        response = match.group(1).strip()

    response = re.sub(r'(\b\w+\b\s+\b\w+\b\s+\b\w+\b.*)\s+\1', r'\1', response, flags=re.IGNORECASE)
    response = re.sub(r'\b(ifnmg\s+instituto\s+federal.*?)\b', r'IFNMG', response, flags=re.IGNORECASE)
    response = re.sub(r'\s+', ' ', response).strip()
    
    return response.strip()

print("\n✅ --- Célula 1 (Setup) Concluída: Funções e paths carregados. --- ✅")

Adicionado ao sys.path: C:\Users\extre\RAG-para-pdfs\src

✅ --- Célula 1 (Setup) Concluída: Funções e paths carregados. --- ✅


In [2]:
print("\n--- INICIANDO PIPELINE RAG COMPLETO (VERSÃO NOTEBOOK) ---")

# --- FASE 0.5: CARREGAR DICIONÁRIOS ---
DATA_INPUT_DIRECTORY = os.path.join(PROJECT_ROOT_PATH, "data", "input")
mapas_normalizacao = carregar_dicionarios(DATA_INPUT_DIRECTORY)
acronyms = mapas_normalizacao["acronyms"]
standardization = mapas_normalizacao["standardization"]

# --- FASE 1: CARREGAMENTO DOS DADOS ---
JSONL_DIRECTORY = os.path.join(PROJECT_ROOT_PATH, "data", "output")
all_documents, report = load_and_process_jsons(JSONL_DIRECTORY)
if not all_documents:
    print("!!! ERRO: Nenhum documento foi carregado. Abortando.")
    raise ValueError("Nenhum documento foi carregado.")
else:
    print(f"\n--- Fase 1 Concluída: {len(all_documents)} documentos (blocos) carregados. ---")


# --- FASE 2: VECTOR STORE ---
embedding_model = get_embedding_model()
index_name = "faiss_index_v_smart_chunk" 
vector_store = get_vector_store(
    documents=all_documents,
    embedding_model=embedding_model,
    index_path=index_name 
)

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 12}
)
print(f"Retriever configurado para buscar k=12 documentos.")
print(f"\n--- Fase 2 Concluída: Retriever está pronto! ---")


# --- FASE 3: CARREGAR O LLM ---
llm = get_llm()
if llm is None:
    print("!!! ERRO: O LLM falhou ao carregar (é 'None'). Abortando.")
    raise ValueError("LLM falhou ao carregar.")
else:
    print(f"\n--- Fase 3 Concluída: LLM (Gemma) está pronto! ---")


# --- FASE 4: MONTAR O PIPELINE DE RAG ---
rag_chain = create_rag_chain(retriever, llm)
print(f"\n--- Fase 4 Concluída: Pipeline RAG está pronto! ---")


# --- FASE 5: TESTE DE PERGUNTAS E RESPOSTAS ---
print("\n\n--- INICIANDO FASE 5: TESTE DE PERGUNTAS E RESPOSTAS ---")
perguntas_teste = [
    "Qual é a frequencia necessario para ser aprovado em uma materia?",
    "Quais são os objetivos do curso de Ciência da Computação?",
    "Qual a forma de ingresso no curso de Ciência da Computação?",
    "Qual é a carga horaria semanal do estágio?",
    "Qual carga horaria máxima das Atividades Complementares no curso?",
    "Qual a carga horaria do curso?",
    "Quais são os professores que podem ser orientadores do TCC?",
    "Qual é o tempo máximo permitido para que um estudante conclua o curso de Ciência da Computação?",
    "Como o aluno sabe se foi aprovado ou reprovado no TCC?",
    "Qual o criterio para aprovação em uma materia?"
]

resultados_finais = {} 

for i, pergunta_bruta in enumerate(perguntas_teste):
    print(f"\n--- Processando Pergunta {i+1}/{len(perguntas_teste)} ---")
    print(f"PERGUNTA (Bruta): {pergunta_bruta}")
    
    # --- NORMALIZA A PERGUNTA ---
    pergunta_normalizada = normalize_query(pergunta_bruta, acronyms, standardization)
    print(f"PERGUNTA (Normalizada): {pergunta_normalizada}")
    
    # 1. Executa o RAG com a pergunta normalizada
    resposta_bruta = rag_chain.invoke(pergunta_normalizada)
    
    # 2. Limpa a resposta
    resposta_limpa = clean_model_response(resposta_bruta)
    print(f"\nRESPOSTA (Gemma):\n{resposta_limpa}")
    
    # 3. Mostra as fontes (usando a pergunta normalizada para o debug)
    documentos_fonte = retriever.invoke(pergunta_normalizada)
    print("\nFONTES (Documentos encontrados pelo FAISS):")
    for j, doc in enumerate(documentos_fonte):
        print(f"  Fonte {j+1}: {doc.metadata.get('source_file')} (Página: {doc.metadata.get('pagina')})")
        print(f"    Trecho (Busca): {doc.page_content[:100]}...")
        print(f"    Trecho (Resposta):    {doc.metadata.get('texto_bruto_resposta', 'N/A')[:100]}...\n")
    
    resultados_finais[pergunta_bruta] = resposta_limpa
    
    print("--------------------------------------------------")

print("\n--- Teste de Perguntas Concluído! ---")


--- INICIANDO PIPELINE RAG COMPLETO (VERSÃO NOTEBOOK) ---
-> Dicionários de normalização carregados para o Testador RAG.
--- [RAG Loader v_SMART_CHUNK] Iniciando Tarefa 1: Carregamento e Fatiamento ---
 [RAG Loader] Processando e fatiando 2 arquivos .jsonl...
 [RAG Loader] Processando arquivo: Estágio.jsonl
   -> Concluído. 28 chunks semânticos criados.
 [RAG Loader] Processando arquivo: PPCBCC2019.jsonl
   -> Concluído. 206 chunks semânticos criados.
--- [RAG Loader] Processamento Concluído ---
Total de 234 documentos (chunks agrupados) prontos para o vector store.

--- Fase 1 Concluída: 234 documentos (blocos) carregados. ---
🔤 Carregando modelo de embeddings: neuralmind/bert-base-portuguese-cased


No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.
C:\Users\extre\anaconda3\envs\ia_projeto\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


  [RAG VectorStore] Modelo de embedding carregado (na CPU).
--- [RAG VectorStore] Iniciando Tarefa 2: Criação do Vector Store ---
  [RAG VectorStore] Carregando índice FAISS existente de: 'faiss_index_v_smart_chunk'
    -> Índice FAISS carregado com sucesso.
--- [RAG VectorStore] Tarefa 2 Concluída. Vector Store está pronto. ---
Retriever configurado para buscar k=12 documentos.

--- Fase 2 Concluída: Retriever está pronto! ---
--- [RAG ModelSetup] Iniciando Tarefa 3: Carregando LLM: 'google/gemma-2b-it' ---
  [RAG ModelSetup] Tokenizador carregado.
  [RAG ModelSetup] Configurando quantização de 8-bit COM offload para CPU...


Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
C:\Users\extre\RAG-para-pdfs\src\rag_pipeline\model_setup.py:56: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm_pipeline = HuggingFacePipeline(pipeline=pipe)


  [RAG ModelSetup] Modelo LLM carregado (em 8-bit, com offload GPU/CPU).
--- [RAG ModelSetup] Tarefa 3 Concluída. LLM está pronto. ---

--- Fase 3 Concluída: LLM (Gemma) está pronto! ---
--- [RAG Chain] Iniciando Tarefa 4: Montando o RAG Chain ---
--- [RAG Chain] Tarefa 4 Concluída. O RAG Chain está pronto. ---

--- Fase 4 Concluída: Pipeline RAG está pronto! ---


--- INICIANDO FASE 5: TESTE DE PERGUNTAS E RESPOSTAS ---

--- Processando Pergunta 1/10 ---
PERGUNTA (Bruta): Qual é a frequencia necessario para ser aprovado em uma materia?
PERGUNTA (Normalizada): qual e a frequencia necessario para ser aprovado em uma disciplina

RESPOSTA (Gemma):
A frequência precisa ser igual ou superior a 75% do total de aulas da disciplina.

FONTES (Documentos encontrados pelo FAISS):
  Fonte 1: None (Página: None)
    Trecho (Busca): a avaliacao do desempenho do aluno constituira elemento fundamental para acompanha- mento e redireci...
    Trecho (Resposta):    a avaliacao do desempenho do aluno cons

In [3]:
# --- RESUMO FINAL (Limpo) ---
print("\n\n" + "="*60)
print("--- RESUMO FINAL (PERGUNTAS E RESPOSTAS DIRETAS) ---")
print("="*60 + "\n")

if not resultados_finais:
    print("Nenhum resultado para exibir.")
else:
    for i, (pergunta, resposta) in enumerate(resultados_finais.items()):
        print(f"--- Pergunta {i+1} ---")
        print(f"P: {pergunta}")
        print(f"R: {resposta}\n")

print("="*60)



--- RESUMO FINAL (PERGUNTAS E RESPOSTAS DIRETAS) ---

--- Pergunta 1 ---
P: Qual é a frequencia necessario para ser aprovado em uma materia?
R: A frequência precisa ser igual ou superior a 75% do total de aulas da disciplina.

--- Pergunta 2 ---
P: Quais são os objetivos do curso de Ciência da Computação?
R: Os objetivos do curso de ciência da computação do IFNMG do norte de minas gerais - campus montes claros são: - formar profissionais com competencia para promover o desenvolvimento cientifico teorias, metodos, linguagens, modelos, dentre outras e tecnologico computacao na regiao norte de minas gerais; - desenvolver no futuro profissional a capacidade de resolver problemas computacionais complexos, desenvolver novos algoritmos, sistemas, provas, metodos e metricas relacionadas a computacao; - formar profissionais capazes de empreender na criacao de novos negócios e favorecerem o desenvolvimento regional; - propiciar condicoes para a formacao de lideranca, capacitando-os para o dese

## Avaliação final


O pipeline RAG teve um desempenho bem-sucedido. Ele **respondeu corretamente a todas as perguntas** e conseguiu extrair as respostas de **trechos relevantes** dos documentos. O mecanismo funcionou identificando os *chunks* dos documentos separadamente e, em seguida, comparando o conteúdo dos blocos de cada *chunk* para formular a resposta, assim não possuindo confusão entre os documentos.